In [1]:
!pip install pandas faiss-cpu sentence-transformers transformers torch


In [2]:
import pandas as pd

df = pd.read_csv('Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv')

# Inspect columns 
print(df.columns)
print(df.head())

# we will use 'Customer_Query' as text and 'Response' as answer
queries = df['instruction'].astype(str).tolist()
responses = df['response'].astype(str).tolist()




Index(['flags', 'instruction', 'category', 'intent', 'response'], dtype='object')
   flags                                        instruction category  \
0      B   question about cancelling order {{Order Number}}    ORDER   
1    BQZ  i have a question about cancelling oorder {{Or...    ORDER   
2   BLQZ    i need help cancelling puchase {{Order Number}}    ORDER   
3     BL         I need to cancel purchase {{Order Number}}    ORDER   
4  BCELN  I cannot afford this order, cancel purchase {{...    ORDER   

         intent                                           response  
0  cancel_order  I've understood you have a question regarding ...  
1  cancel_order  I've been informed that you have a question ab...  
2  cancel_order  I can sense that you're seeking assistance wit...  
3  cancel_order  I understood that you need assistance with can...  
4  cancel_order  I'm sensitive to the fact that you're facing f...  


In [3]:
!pip install tf-keras keras==2.11.0


  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-2.11.0-py2.py3-none-any.whl.metadata (1.4 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached tensorflow-2.18.1-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
INFO: pip is looking at multiple versions of tensorflow-intel to determine which version is compatible with other requirements. This could take a while.
  Using cached tf_keras-2.17.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached tensorflow-2.17.1-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.17.1-cp312-cp312-w

ERROR: Cannot install keras==2.11.0 and tensorflow because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [5]:
!pip install tf-keras


^C
Note: you may need to restart the kernel to use updated packages.


  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
Using cached tf_keras-2.19.0-py3-none-any.whl (1.7 MB)
Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)


In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')  

query_embeddings = model.encode(queries, show_progress_bar=True)


Batches:   0%|          | 0/840 [00:00<?, ?it/s]

In [7]:
import faiss

dimension = query_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  

index.add(np.array(query_embeddings).astype('float32'))


In [8]:
def get_response(user_input, top_k=1):
    user_emb = model.encode([user_input])
    D, I = index.search(np.array(user_emb).astype('float32'), top_k)
    return [responses[i] for i in I[0]]


In [9]:
!pip install huggingface_hub[hf_xet]


In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load GPT2 for generation
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


def generate_answer(context, question):
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = gpt2_tokenizer.encode(input_text, return_tensors='pt')
    outputs = gpt2_model.generate(inputs, max_length=100)
    answer = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer



In [12]:
def get_response(user_input, top_k=1):
    user_emb = embed_model.encode([user_input])
    D, I = index.search(np.array(user_emb).astype('float32'), top_k)
    return [responses[i] for i in I[0]]



In [13]:
def chatbot():
    print("Customer Service Chatbot (type 'exit' to quit)")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        retrieved = get_response(user_input)[0]

        print("Bot:", retrieved)

if __name__ == "__main__":
    chatbot()


Customer Service Chatbot (type 'exit' to quit)
Bot: Honored to assist! I'm clued in that you would like to chat with a person. Our customer support team is here to assist you. Please allow me a moment to connect you with one of our dedicated representatives who will be happy to help you further.
Bot: Always good to connect! I'm attuned to the fact that you're looking for someone to assist you. Our team of dedicated human agents is here to help. Please allow me a moment to connect you with the right person who can address your needs.
Bot: Always good to connect! I'm attuned to the fact that you're looking for someone to assist you. Our team of dedicated human agents is here to help. Please allow me a moment to connect you with the right person who can address your needs.
